# PROYECTO DE CONSOLIDACIÓN: APRENDIZAJE SUPERVISADO

## [EJERCICIO 1] Explicación
Bienvenido al año 2912, donde se necesitan tus habilidades de ciencia de datos para resolver un misterio cósmico. Hemos recibido una transmisión desde cuatro años luz de distancia y las cosas no pintan bien.

La nave espacial Alcón Milenario fue un transatlántico de pasajeros interestelar lanzado hace un mes. Con casi 13.000 pasajeros a bordo, la nave emprendió su viaje inaugural transportando emigrantes de nuestro sistema solar a tres exoplanetas recientemente habitables que orbitan estrellas cercanas.

Mientras rodeaba Alpha Centauri en ruta hacia su primer destino, el tórrido 55 Cancri E, la desprevenida nave espacial Alcón Milenario chocó con una anomalía del espacio-tiempo escondida dentro de una nube de polvo. Lamentablemente, tuvo un destino similar al de su homónimo de 1000 años antes. Aunque la nave permaneció intacta, ¡casi la mitad de los pasajeros fueron transportados a una dimensión alternativa!

Para ayudar a las tripulaciones de rescate y recuperar a los pasajeros perdidos, debe predecir qué pasajeros fueron transportados por la anomalía utilizando los registros recuperados del sistema informático dañado de la nave espacial.

¡Ayuda a salvarlos y cambia la historia!

###### a) Realiza un análisis exploratorio de los datos para entender la distribución de las diferentes variables.
###### b) Procesa los datos para:
###### - Detectar y tratar los posibles valores faltantes.
###### - Detectar y tratar outliers.
###### c) Utiliza las técnicas vistas en clase para detectar las variables más importante con respecto a la variable objetivo (Transported)
###### d) Aplica alguna de las técnicas vistas en clase para determinar qué pasajeros fueron transportados y cuales no.
###### e) Mejora los resultados optimizando los parámetros.
###### f) Compara los resultados obtenidos utilizando modelos de ensemble para resolver el problema.
###### g) Selecciona el mejor modelo de los testeados en los apartados anteriores.

###### Deberás entrenar y validar el modelo seleccionado sobre el dataset "spaceship_alcon_train.csv" y aplicar posteriormente este modelo para predecir sobre el dataset "spaceship_alcon_test.csv".
###### Deberás explicar todos los pasos que vayas dando en la resolución del problema.

## [EJERCICIO 1] Resolución

### Importar Librerías y funciones:

In [171]:
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

import numpy as np
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

### Importar Dataset:

In [172]:
dataset=pd.read_csv("spaceship_alcon_train.csv")

Lo primero de todo, analizamos el dataset del conjunto de entrenamiento, y vemos que este proporciona una variedad de datos sobre los pasajeros de la nave espacial, incluyendo:

- **PassengerId**: Identificador único para cada pasajero.
- **HomePlanet**: El planeta/ubicación de origen del pasajero.
- **CryoSleep**: Indica si el pasajero eligió estar en "cryosueño" durante el viaje.
- **Cabin**: El número de la cabina del pasajero.
- **Destination**: El destino planetario del pasajero.
- **Age**: La edad del pasajero.
- **VIP**: Indica si el pasajero pagó por servicio VIP en la nave.
- **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck**: Gastos del pasajero en diferentes servicios a bordo.
- **Name**: Nombre del pasajero.
- **Transported**: Variable objetivo que indica si el pasajero fue transportado a una dimensión alternativa (True/False).

El siguiente paso es realizar un **Análisis Exploratorio de Datos (EDA)** para entender mejor la distribución de las variables, identificar valores faltantes, y observar posibles outliers. Esto incluirá estadísticas descriptivas, distribuciones de variables, y correlaciones potenciales entre variables. Para ello, vamos a iniciar con un resumen estadístico del conjunto de entrenamiento para identificar patrones iniciales, valores atípicos y valores faltantes.

### a) Realiza un análisis exploratorio de los datos para entender la distribución de las diferentes variables.

In [173]:
dataset.head()

In [174]:
dataset.shape

In [175]:
dataset.describe(include= object)

In [176]:
dataset.describe()

In [177]:
dataset.isna().sum()

In [178]:
#pip install ydata_profiling 
#pip install ipywidgets

In [179]:
from ydata_profiling import ProfileReport

report = ProfileReport(dataset)
report

El resumen estadístico y la información sobre valores faltantes nos revelan varios puntos clave sobre el conjunto de entrenamiento:

1.- **Valores Faltantes**: Hay valores faltantes en varias columnas, incluyendo `HomePlanet` (201 faltantes), `CryoSleep` (217), `Cabin` (199), `Destination` (182), `Age` (179), `VIP` (203), `RoomService` (181), `FoodCourt` (183), `ShoppingMall` (208), `Spa` (183), `VRDeck` (188), y `Name` (200). Es crucial tratar estos valores faltantes antes de modelar.


2.- **Variables Categóricas**:
- `HomePlanet` tiene tres valores únicos, siendo `Earth` el más común, seguido de `Europa` y `Marte`.
- `Cabin` muestra una gran variedad, lo que sugiere que podría necesitar un tratamiento especial para su uso en el modelado, similar al que tendremos que utilizar con `Name` y `PassengerID`.
- `CryoSleep`: Una cantidad significativa de pasajeros (5439) no estuvo en CryoSleep, mientras que 3037 sí lo estuvieron.
- `Destination`: La mayoría de los pasajeros se dirigían a TRAPPIST-1e, seguido por 55 Cancri e y, en menor medida, PSO J318.5-22
- `VIP`: Muy pocos pasajeros adquirieron el status de VIP.


3.- **Variables Numéricas**:
- `Age` varía entre 0 y 79 años, con una media de aproximadamente 28,8 años.
- Los gastos en servicios a bordo (`RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`) muestran una gran variabilidad, con muchos valores en 0 (posiblemente indicando pasajeros que no usaron esos servicios) y algunos valores extremadamente altos, lo que sugiere la presencia de outliers.


4.- **Variable Objetivo (`Transported`)**: La variable objetivo es binaria, indicando si un pasajero fue transportado a una dimensión alternativa. La distribución es relativamente balanceada con una ligera mayoría de casos `True`.

In [180]:
dataset["HomePlanet"].value_counts()

In [181]:
dataset["CryoSleep"].value_counts()

In [182]:
dataset["Destination"].value_counts()

In [183]:
dataset["VIP"].value_counts()

In [184]:
dataset.Cabin.unique()

In [185]:
list = ["PassengerId","HomePlanet","Cabin","Destination","Name"]
dataset_num= dataset.drop(list, axis = 1)
correlation_matrix = dataset_num.corr().round(2)
plt.figure(figsize=(10,7))
#sns.heatmap(data=correlation_matrix, annot=True)
sns.heatmap(correlation_matrix, annot=True, fmt=".1f")

In [186]:
list = ["PassengerId","HomePlanet","Cabin","Destination","Name","VIP","Transported","Age"]
dataset_cryo= dataset.drop(list, axis = 1)
correlation_matrix = dataset_cryo.corr().round(2)
plt.figure(figsize=(10,7))
sns.heatmap(correlation_matrix, annot=True, fmt=".1f")

In [187]:
sns.countplot(x = "Transported", hue="HomePlanet", data = dataset)

In [188]:
sns.countplot(x = "Transported", hue="Destination", data = dataset)

In [189]:
sns.catplot(x= "VIP", kind = "count", data = dataset)

In [190]:
sns.catplot(x= "CryoSleep", kind = "count", data = dataset)

In [191]:
sns.catplot(x= "Transported", kind = "count", data = dataset)

In [192]:
sns.boxplot(x = "CryoSleep",y= "Age", data = dataset)

In [193]:
sns.boxplot(x="Transported", y= "Age", data = dataset)

In [194]:
sns.boxplot(x="HomePlanet", y="Age", data = dataset)

In [195]:
sns.boxplot(x="VIP", y="Age", data = dataset)

In [196]:
gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'] # Variables de gastos

# Revisamos las distribuciones de los gastos
fig, axs = plt.subplots(5, 1, figsize=(18, 12))
for var, ax in zip(gastos, axs.flat):
    sns.boxplot(x=dataset[var], ax=ax)
    ax.set_title(f'Boxplot de {var}')

plt.tight_layout()
plt.show()

In [197]:
sns.displot(kind="kde", data= dataset, x = "Age", hue="HomePlanet", fill = "True")

In [198]:
sns.displot(kind="kde", data= dataset, x = "Age", hue="Destination", fill = "True")

**Implicaciones para la Limpieza de Datos y el Tratamiento de Outliers**

Hacemos gráficas aparte (para tenerlas incluidas en el dataset fuera del report) así como un mapa de correlaciones, para tenerlas más a mano que en el `report`, por todo ello, podemos sacar las siguientes conclusiones del conjunto de entrenamiento sobre el tratamiento de outliers y faltantes que tenemos que hacer: 

**Outliers en las Variables de Gastos:**
- Los boxplots para las variables de gastos (`RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`) muestran claramente la presencia de outliers, con muchos valores extremadamente altos en comparación con la mayoría de los datos. Aunque, dada la naturaleza de estos datos, la presencia de outliers no necesariamente indica datos incorrectos o anomalías; podrían representar simplemente pasajeros que gastaron mucho más que otros.

**Correlaciones con CryoSleep y Tratamiento de Faltantes:**
- Con el mapa de correlaciones podemos observar lo siguiente:
  - **CryoSleep tiene correlaciones negativas con todas las variables de gastos**, lo cual es consistente con la idea que teníamos de que los pasajeros en CryoSleep no gastarían en servicios a bordo, puesto que no pueden, al no estar "despiertos". Estas correlaciones negativas apoyan la hipótesis de que los valores cero y posiblemente algunos valores faltantes en las variables de gastos podrían estar relacionados con pasajeros en CryoSleep.
  - **VIP muestra correlaciones positivas moderadas con algunas variables de gastos**, lo que sugiere que los pasajeros VIP tienden a gastar más en ciertos servicios, lo cual podemos tenerlo en cuenta para los outliers.

### b) Procesa los datos para:

Para abordar el tratamiento de valores faltantes y outliers, hemos procedido de la siguiente manera:

1. **Valores Faltantes:**
   - Para los pasajeros en CryoSleep (`CryoSleep` es True), imputaremos los valores faltantes de las variables numéricas relacionadas con los gastos (`RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`) con 0, ya que estos pasajeros no tendrían la oportunidad de gastar, y para el resto, imputaremos la mediana.
   - Para la edad (`Age`), utilizaremos la mediana como técnica de imputación que considere la distribución de esta variable, para todos los pasajeros independientemente de su estado de CryoSleep.
   - Para las variables categóricas con valores faltantes y para `CryoSleep` y `VIP` donde no se especifique, aplicaremos la moda (el valor más frecuente) como técnica de imputación.

2. **Outliers:**
   - Una vez analizado el dataset no consideramos que la presencia de outliers sean datos incorrectos o anomalías por la naturaleza del dataset, por ello, vamos a proceder con el tratamiento de faltantes para no de outliers. 

#### - Detectar y tratar los posibles valores faltantes.

In [199]:
dataset.isna().mean()*100

In [200]:
dataset.dtypes

- Imputación para variables categóricas con la moda

In [201]:
rev_null=['HomePlanet','CryoSleep','Cabin','Destination','VIP']
dataset[rev_null]=dataset[rev_null].replace({np.nan:dataset['HomePlanet'].mode(),
                                                np.nan:dataset['CryoSleep'].mode(),
                                                np.nan:dataset['Cabin'].mode(),
                                                np.nan:dataset['Destination'].mode(),
                                                np.nan:dataset['VIP'].median()})

- Imputación para la variable numérica con la mediana.

In [202]:
dataset["Age"].fillna(dataset["Age"].median(), inplace=True)

In [203]:
dataset.isna().sum()

Del análisis gráfico se deduce que los que están en criosueño no generan ningún consumo, por lo que se aplica el valor cero a las filas de criosueño = True

In [204]:
list_num_null= ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

for i in list_num_null:
    dataset[i] = np.where(dataset['CryoSleep'] == True, 0, dataset[i])   

In [205]:
dataset.isna().sum()

In [206]:
dataset[list_num_null]= dataset[list_num_null].replace({np.nan:dataset["RoomService"].median(),
                                                        np.nan:dataset["FoodCourt"].median(),
                                                        np.nan:dataset["ShoppingMall"].median(),
                                                        np.nan:dataset["Spa"].median(),
                                                        np.nan:dataset["VRDeck"].median()})

In [207]:
dataset.isna().sum()

### Preprocesamiento de Datos

In [208]:
dataset_filter = dataset.drop(["PassengerId","Name","Cabin"], axis = 1)

In [209]:
dataset_filter = pd.get_dummies(dataset_filter, dtype=int, drop_first = True)

In [210]:
dataset_filter

In [211]:
X = dataset_filter.drop("Transported", axis = 1)
y = dataset_filter["Transported"]

Dividir el data set en conjunto de entrenamiento y conjunto de test

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

Normalizamos variables

In [213]:
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

Utilizamos un modelo de Random Forest para ver que variables tienen más peso.

In [214]:
rf = RandomForestClassifier(criterion = 'entropy', n_estimators = 10)

In [215]:
rf.fit(X_train,y_train)
rf.feature_importances_

In [216]:
#plt.figure(figsize=(10,15))
plt.barh(X.columns, rf.feature_importances_)

Selección de las variables predictoras con mayor importancia:

In [217]:
X = X[[ "Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","CryoSleep_True"]]

División en entrenamiento y test:

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

Normalización de variables

In [219]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Definimos distintos modelos de clasificación y calculamos las métricas para ver cuál se comporta mejor:

In [220]:
reg = LogisticRegression()
svm = SVC()
nb = GaussianNB()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=7)

Entrenamos los distintos modelos:

In [221]:
reg.fit(X_train,y_train.ravel())
svm.fit(X_train,y_train.ravel())
nb.fit(X_train,y_train.ravel())
dt.fit(X_train,y_train.ravel())
knn.fit(X_train, y_train.ravel())

Calculamos el accuracy en el conjunto de validación (test) y visualizamos las métricas acc en un gráfico de barras:

In [222]:
accuracyList = []
algorithm = []

#Cálculo acc y almacena en lista
accuracyList.append(reg.score(X_test,y_test))
accuracyList.append(svm.score(X_test,y_test))
accuracyList.append(nb.score(X_test,y_test))
accuracyList.append(dt.score(X_test,y_test))
accuracyList.append(knn.score(X_test,y_test))

algorithm.append("Logistic Regression")
algorithm.append("Support Vector Machine")
algorithm.append("Naive Bayes")
algorithm.append("Decision Tree")
algorithm.append("KNN")

#Visualización gráficas:
f,ax = plt.subplots(figsize = (15,7))
sns.barplot(x=accuracyList,y=algorithm,palette = sns.cubehelix_palette(len(accuracyList)))
plt.xlabel("Accuracy")
plt.ylabel("Classifier")
plt.title('Classifier Accuracy')
plt.show()

Vamos ahora a predecir sobre el conjunto de test y calcular la matriz de confusión:

In [223]:
predict_list = []
y_pred_LR = reg.predict(X_test)
y_pred_SVM = svm.predict(X_test)
y_pred_NB = nb.predict(X_test)
y_pred_DT = dt.predict(X_test)
y_pred_KNN = knn.predict(X_test)

In [224]:
#Matriz de confusión:
cmLR = confusion_matrix(y_test,y_pred_LR)
cmSVM = confusion_matrix(y_test,y_pred_SVM)
cmNB = confusion_matrix(y_test,y_pred_NB)
cmDT = confusion_matrix(y_test,y_pred_DT)
cmKNN = confusion_matrix(y_test,y_pred_KNN)

Visualizamos las distintas matrices para cada uno de los modelos entrenados:

In [225]:
width = 25
height = 5

f, ax = plt.subplots(1, 5)
f.set_size_inches(width, height)

# Logistic Regression
sns.heatmap(cmLR,annot = True,linewidths=0.5,
            linecolor="red",fmt = ".0f",ax=ax[0])
# Support Vector Machines
sns.heatmap(cmSVM,annot = True,linewidths=0.5,
            linecolor="red",fmt = ".0f",ax=ax[1])
# Naive Bayes
sns.heatmap(cmNB,annot = True,linewidths=0.5,
            linecolor="red",fmt = ".0f",ax=ax[2])
# Decision Tree
sns.heatmap(cmDT,annot = True,linewidths=0.5,
            linecolor="red",fmt = ".0f",ax=ax[3])
# KNN
sns.heatmap(cmKNN,annot = True,linewidths=0.5,
            linecolor="red",fmt = ".0f",ax=ax[4])

f.show()

In [226]:
print(accuracy_score(y_test,y_pred_LR))
print(accuracy_score(y_test,y_pred_SVM))
print(accuracy_score(y_test,y_pred_NB))
print(accuracy_score(y_test,y_pred_DT))
print(accuracy_score(y_test,y_pred_KNN))

### Aplicamos métodos Ensemble: modelo Baggin

In [227]:
# Definir los estimadores base a probar
base_estimators = {
    "Decision Tree": DecisionTreeClassifier(max_depth=None),
    "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Support Vector Marchine": SVC(),
    "Naive Bayes": GaussianNB(),
}

In [228]:
# Lista para guardar los resultados
results = []

for name, estimator in base_estimators.items():
    for n_estimators in [5, 10, 20, 50]:
        # Crear y entrenar el modelo de bagging
        model = BaggingClassifier(base_estimator=estimator, n_estimators=n_estimators, random_state=42)
        model.fit(X_train, y_train)
        
        # Realizar predicciones y calcular la precisión
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        train_accuracy = accuracy_score(y_train, y_pred_train)
        test_accuracy = accuracy_score(y_test, y_pred_test)
        
        # Guardar los resultados
        results.append({
            "Base Estimator": name,
            "N Estimators": n_estimators,
            "Train Accuracy": train_accuracy,
            "Test Accuracy": test_accuracy
        })

# Convertir los resultados en un DataFrame para una mejor visualización
import pandas as pd
results_df = pd.DataFrame(results)

In [229]:
results_df.sort_values("Test Accuracy", ascending = False)

### Aplicamos métodos Ensemble: modelo Adaboost

In [230]:
# Definir los estimadores base a probar
estimators = {
    "Decision Tree": DecisionTreeClassifier(max_depth=None),
    "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42)
}

In [231]:
# Lista para guardar los resultados
results_ada = []

for name, estim in estimators.items():
    for n_estimators in [5, 10, 20, 50]:
        # Crear y entrenar el modelo de AdaBoost
        model = AdaBoostClassifier(estimator=estim, n_estimators=n_estimators, random_state=42)
        model.fit(X_train, y_train)
        
        # Realizar predicciones y calcular la precisión
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        train_accuracy = accuracy_score(y_train, y_pred_train)
        test_accuracy = accuracy_score(y_test, y_pred_test)
        
        # Guardar los resultados
        results_ada.append({
            "Base Estimator": name,
            "N Estimators": n_estimators,
            "Train Accuracy": train_accuracy,
            "Test Accuracy": test_accuracy
        })

# Convertir los resultados en un DataFrame para una mejor visualización
import pandas as pd
results_ada_df = pd.DataFrame(results_ada)

In [232]:
results_ada_df.sort_values("Test Accuracy", ascending = False)

Analizando los resultados, teniendo en cuenta los resultados de train y test, se decide elegir el Árbol de decisión con 20 estimadores por tener unos resultados muy similares al de 50 estimadores pero es menos complejo y consume menos recursos. También tenemos en cuenta que aplicando Adaboost, el mejor es, de nuevo, el Árbol de decisión con 20 estimadores, pero los resultados son algo peores que con el de Baggin. Dicho esto, procedemos a aplicar el que consideremos el mejor modelo.

### Aplicamos el mejor estimador al conjunto de test: Decision Tree N Estimators: 20

In [233]:
dataset_test=pd.read_csv("spaceship_alcon_test.csv")

In [234]:
dataset_test.head()

In [235]:
dataset_test_filter = dataset_test[["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","CryoSleep"]]

In [236]:
dataset_test_filter.isna().sum()

In [237]:
dataset_test_filter.CryoSleep.fillna(dataset_test_filter.CryoSleep.value_counts().idxmax(),inplace = True) 

In [238]:
dataset_test_filter["Age"].fillna(dataset_test_filter["Age"].median(), inplace = True)

In [239]:
list_num_null= ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

for i in list_num_null:
    dataset_test_filter[i] = np.where(dataset_test_filter['CryoSleep'] == True, 0, dataset_test_filter[i])   

In [240]:
dataset_test_filter[list_num_null]= dataset_test_filter[list_num_null].replace({np.nan:dataset["RoomService"].median(),
                                                        np.nan:dataset["FoodCourt"].median(),
                                                        np.nan:dataset["ShoppingMall"].median(),
                                                        np.nan:dataset["Spa"].median(),
                                                        np.nan:dataset["VRDeck"].median()})

In [241]:
dataset_test_filter.isna().sum()

In [242]:
X = dataset_test_filter

In [243]:
sc_X = MinMaxScaler()
X = sc_X.fit_transform(X)

In [244]:
estimador = DecisionTreeClassifier(max_depth=None)

In [245]:
baggModel = BaggingClassifier(base_estimator = estimador, n_estimators=20, random_state=42)

In [246]:
baggModel.fit(X_train, y_train)
y_pred_test = baggModel.predict(X)

In [247]:
dataset_test_filter["Transported"] = y_pred_test
dataset_test_filter

## [EJERCICIO 2]

La empresa está desarrollando una aplicación a modo TripAdvisor en la que los usuarios van a poder valorar una serie de restaurantes en base a su experiencia. Para que los restaurantes se adieran a la aplicación y quieran compartirla con sus clientes, estos desean saber, en base a sus características, que valoración pueden tener. Para ayudarles, hay que tratar de predecir la valoración (variable rate) que obtendría cada restaurante en base a las variables conocidas:

- name: Nombre del restaurante.
- online_order:¿Aceptan pedidos en línea? (Verdadero/Falso)
- book_table: ¿Podemos reservar mesa en el restaurante?
- rate: Valoración que dan los usuarios en la aplicación (variable objetivo).
- votes: Número de valoraciones realizadas a un restaurante.
- location: Ubicación del restaurante.
- rest_type: Tipo de restaurante.
- cuisines: Tipo de cocina.
- Cost2plate: Coste aproximado de dos platos.
- Type: Categoría del restaurante.

### La empresa solicita el siguiente informe:
#### a) Ver el número de restaurantes por cada una de las localizaciones.
#### b) Conocer el número de restaurantes que permiten y no permiten pedidos online y aquellos que permiten reservar mesa.
#### c) Queremos conocer, mediante una visualización, cual es la distribución de la valoracion de los usuarios en función de si el restaurante permite o no pedidos online.
#### d) Queremos conocer, mediante una visualización, cual es la distribución de la valoracion de los usuarios en función de si el restaurante permite o no reservar mesa.
#### e) ¿Cual de las dos variables analizadas en los apartados d y C crees que sería más determinante de cara a un modelo de Machine Learning? ¿Por qué?
#### f) Realiza las transformaciones que creas necesarias en las variables, explicando cada una de ellas, de cara a crear un modelo de regresión que sea capaz de predecir la valoración de los usuarios.
#### g) Aplica un modelo de Regresión de los vistos en clase para tratar de predecir la valoración de los usuarios. Trata de mejorar los resultados aplicando otros modelos. (Regresión Lineal Múltiple, Árboles de Regresión o Random Forest Regressor)
#### h) Selecciona el mejor modelo basándote en la métrica del RMSE.

##### Deberás entrenar y validar el modelo seleccionado sobre el dataset "app_restaurantes.csv".

##### Deberás explicar todos los pasos que vayas dando en la resolución del problema.

In [248]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier

In [249]:
restaurants = pd.read_csv("app_restaurantes_clean.csv")

In [250]:
restaurants.head()

In [251]:
restaurants.describe()

In [252]:
restaurants.isna().sum()

In [253]:
from ydata_profiling import ProfileReport

report = ProfileReport(restaurants)
report

### a) Ver el número de restaurantes por cada una de las localizaciones

In [254]:
restaurants.groupby("location")[["name"]].count() #Número de restaurantes por cada localización

Otra forma de ver el número de restaurantes por localización en la que se pueden ver todas las localizaciones es la siguiente:

In [255]:
restaurant_count = restaurants['location'].value_counts().reset_index()
restaurant_count.columns = ['Location', 'Restaurant Count']

In [256]:
for index,  row in restaurant_count.iterrows():
    print(row['Location'], row['Restaurant Count'])

### b) Conocer el número de restaurantes que permiten y no permiten pedidos online y aquellos que permiten reservar mesa

In [257]:
restaurants.groupby("online_order")[["name"]].count() #Número de restaurantes que permiten y no permiten pedidos online

In [258]:
restaurants.groupby("book_table")[["name"]].count() #Número de restaurantes que permiten y no permiten reservar mesa

### c) Conocer, mediante una visualización, cual es la distribución de la valoracion de los usuarios en función de si el restaurante permite o no pedidos online

In [259]:
restaurants.isna().sum()

In [260]:
#Se separan las filas cuyo "rate" es nulo de las que no
restaurants_rate = restaurants[restaurants["rate"].notna()] 
restaurants_rate_null = restaurants[restaurants["rate"].isna()]

In [261]:
def get_rate(field):
    field = str(field)
    num = float(field.split("/")[0])
    
    return num

In [262]:
restaurants_rate["rate"] = restaurants_rate["rate"].map(get_rate)

In [263]:
restaurants_rate.describe()

In [264]:
restaurants = pd.concat([restaurants_rate, restaurants_rate_null])

In [265]:
plt.figure(figsize=(8, 6))
restaurants.groupby(['rate', 'online_order']).size().unstack().plot(kind='bar', stacked=True)
plt.xlabel('Rate')
plt.ylabel('Count')
plt.title('Distribution of Rate by Online Order')
plt.show()

### d) Conocer, mediante una visualización, cual es la distribución de la valoración de los usuarios en función de si el restaurante permite o no reservar mesa

In [266]:
plt.figure(figsize=(8, 6))
restaurants.groupby(['rate', 'book_table']).size().unstack().plot(kind='bar', stacked=True)
plt.xlabel('Book_Table')
plt.ylabel('Count')
plt.title('Distribution of Rate by Book_Table')
plt.show()

### e) ¿Cual de las dos variables analizadas en los apartados c y d crees que sería más determinante de cara a un modelo de Machine Learning? ¿Por qué?

Al observar ambas gráficas, podemos ver que en la segunda gráfica (`rate` vs `book_table`), la variable `book_table` no parece tener un impacto significativo en la distribución de la variable `rate`, ya que las barras se mantienen casi uniformes independientemente de si se dispone de mesa para reservar o no.
Por otro lado, en la primera gráfica (`rate` vs `online_order`), se observa que la variable `online_order` sí tiene un impacto en la distribución de la variable `rate`, porque las barras varían en función de si se permite el pedido en línea o no.
Por lo que: basándonos en las gráficas proporcionadas, parece que *la variable `online_order` sería más determinante* de cara a un modelo de Machine Learning, ya que parece tener un impacto significativo en la variable objetivo `rate`.

Otra forma de analizarlo la cual también nos hace elegir la variable `online_order` como la más determinante es la importante correlación que existe entre `online_order` y `Cost2plate`, como hemos podido ver en el mapa de correlaciones del _report_, haciendo más únicos y significativos de cara a un modelo de Machine Learning los valores de `online_order`.

### f) Realizar las transformaciones que creas necesarias en las variables, explicando cada una de ellas, de cara a crear un modelo de regresión que sea capaz de predecir la valoración de los usuarios

In [267]:
restaurants["rate"] = restaurants["rate"].replace({np.nan:restaurants["rate"].mean()})

In [268]:
restaurants.isna().sum()

In [269]:
list_variables = ['rest_type', 'cuisines', 'location']
restaurants[list_variables] = restaurants[list_variables].replace({np.nan:restaurants['rest_type'].mode()[0],
                                                                            np.nan:restaurants['cuisines'].mode()[0],
                                                                            np.nan:restaurants['location'].mode()[0]})

In [270]:
restaurants["Cost2plate"] = restaurants["Cost2plate"].replace({np.nan:restaurants["Cost2plate"].mean()})

In [271]:
restaurants.isna().sum()

### g) Aplica un modelo de Regresión de los vistos en clase para tratar de predecir la valoración de los usuarios. Trata de mejorar los resultados aplicando otros modelos. (Regresión Lineal Múltiple, Árboles de Regresión o Random Forest Regressor)

Se hace una primera prueba con Regresión Lineal Múltiple quitando las variables categóricas que tienen muchos valores únicos

In [272]:
restaurants_filtered = restaurants[["online_order", "book_table", "votes", "Cost2plate", "Type", "rate"]]

In [273]:
restaurants_filtered = pd.get_dummies(restaurants_filtered, drop_first=True, dtype=int)

In [274]:
restaurants_filtered

In [275]:
X = restaurants_filtered.drop("rate", axis = 1)
y = restaurants_filtered["rate"]

In [276]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [277]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

#### Regresión lineal múltiple

In [278]:
def r2_adj(y_pred, X_test, r2):
    N = y_pred.shape[0]
    p = X_test.shape[1]
    k = (1-r2)
    w = (N-1)/(N-p-1)
    adj_r2 = 1-(k*w)
    
    return adj_r2

In [279]:
X_train_df = pd.DataFrame(X_train, columns = X_train.columns)
X_test_df = pd.DataFrame(X_test, columns = X_train.columns)

In [280]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [281]:
dataset_seleccion = pd.DataFrame() # Se crea vacío y se va rellenando en un bucle
for i in range(1, len(X_train_df.columns)):# Se recorre columna por columna, empieza en 1 porque es el número de columnas que se seleccionan
    sbs = SFS(LinearRegression(), k_features = i, forward = False)
    sbs.fit(np.array(X_train), y_train)
    regression = LinearRegression()
    columnas = [X_train_df.columns[int(index)] for index in sbs.k_feature_idx_] # FYI, Debido a alguna cuestión relacionada con las versiones de Python o Jupyter a algunos nos daba error si poníamos list(in sbs.k_feature_idx_) aunque a otros no nos ocurría ese error.
    regression.fit(X_train_df[columnas], y_train)
    y_pred = regression.predict(X_test_df[columnas])
    r2 = r2_score(y_test, y_pred)
    adj_r2 = r2_adj(y_pred, X_test_df[columnas], r2)
    rmse = mean_squared_error(y_test, y_pred, squared = False)
    mae = mean_squared_error(y_test, y_pred)
    df_result = pd.DataFrame({"variables": [columnas], "r2": adj_r2, "rmse": rmse, "mae": mae})
    dataset_seleccion = pd.concat([dataset_seleccion, df_result], ignore_index = True)
    dataset_seleccion = dataset_seleccion.sort_values(by = 'r2', ascending = False)

In [282]:
dataset_seleccion

Los resultados son muy malos, por lo que se decide añadir las variables "location" y "rest_type" y hacer una codificación binaria para deducir el número de variables que se obtendrían usando One Hot Encoding

In [283]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['online_order', 'book_table', 'location', 'rest_type', 'Type'])

In [284]:
df_rest_limpio2 = encoder.fit_transform(restaurants)

In [285]:
df_rest_limpio2.head()

In [286]:
df_rest_limpio_sin_name = df_rest_limpio2.drop(['name', 'cuisines', 'Unnamed: 0'], axis=1)

In [287]:
df_rest_limpio_sin_name.head()

In [288]:
# Calcular la matriz de correlaciones
correlation_matrix = df_rest_limpio_sin_name.corr().round(2)

# Creamos el gráfico con un tamaño más grande
plt.figure(figsize=(10, 8))

# y crear el heatmap con anotaciones de los valores de correlación
sns.heatmap(data=correlation_matrix, annot=True, annot_kws={"fontsize": 8})

plt.show()

La variable `rate` que posteriormente se usará como Y correlaciona con:
Correlaciones positivas de `rate`: `vote` (0.43), `Cost2plate`(0.37), `restype`(0.15), `book table` (-0.41)
Correlaciones negativas entre otras variables: `Cost2plate` con `book table` (-0.62). `Cost2plate` es inversamente proporcional a `book table`. 
También miramos que `votes`, con `Cost2plate` (0.38), y con  `restype` (0.15) así como `book table`, con `votes` (-0.41).

In [289]:
X = df_rest_limpio_sin_name.drop(['rate'], axis=1)
y = df_rest_limpio_sin_name['rate']

In [290]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [291]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

In [292]:
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

#### Regresión polinómica

In [293]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

In [294]:
def polynomial_regresion(X_te_sc, X_tr_sc, y_tr, y_te, degree):
    poly_features = PolynomialFeatures(degree = degree)
    X_tr_sc_poly = poly_features.fit_transform(X_tr_sc)
    poly_model = LinearRegression()
    poly_model.fit(X_tr_sc_poly, y_tr)
    
    y_tr_predict = poly_model.predict(X_tr_sc_poly)
    r2 = r2_score(y_tr, y_tr_predict)
    rmse = mean_squared_error(y_tr, y_tr_predict, squared = False)

    print("R2 train: {}".format(r2))
    print("RMSE train: {}".format(rmse))

    y_te_predict = poly_model.predict(poly_features.transform(X_te_sc))
    r2 = r2_score(y_te, y_te_predict)
    rmse = mean_squared_error(y_te, y_te_predict, squared = False)

    print("R2 test: {}".format(r2))
    print("RMSE train: {}".format(rmse))
    return y_tr_predict, y_te_predict

In [295]:
y_train_pred, y_teest_pred = polynomial_regresion(X_test_sc, X_train_sc, y_train, y_test, 2) 
#Regresión de grado 2. Con más grados el procesamiento se hace muy pesado para que lo pueda soportar un PC

#### Árbol de decisión

In [296]:
from sklearn.tree import DecisionTreeRegressor
regression = DecisionTreeRegressor(random_state = 0)
regression.fit(X_train_sc, y_train)

In [297]:
y_train = regression.predict(X_train_sc)
y_pred = regression.predict(X_test_sc)

In [298]:
rmse_train = mean_squared_error(y_train, y_train, squared = False)
rmse_test = mean_squared_error(y_test, y_pred, squared = False)

r2_train = r2_score(y_train, y_train)
r2_test = r2_score(y_test, y_pred)

print("RMSE train:", rmse_train)
print("RMSE test:", rmse_test)

print("R2 train:", r2_train)
print("R2 test:", r2_test)

#### Árbol de decisión con Grid Search

In [299]:
from sklearn.model_selection import GridSearchCV

In [300]:
# Se define el modelo de árbol de decisión
model = DecisionTreeRegressor()

# Se definen los hiperparámetros a ajustar
param_grid = {
    'max_depth': [25, 27, 29, 31, 35],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3,]
}

# Se utiliza GridSearchCV para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=15)
grid_search.fit(X_train_sc, y_train)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Utilizamos el modelo con los mejores hiperparámetros para hacer las predicciones
y_train = grid_search.best_estimator_.predict(X_train_sc)
y_pred = grid_search.best_estimator_.predict(X_test_sc)

In [301]:
rmse_train = mean_squared_error(y_train, y_train, squared = False)
rmse_test = mean_squared_error(y_test, y_pred, squared = False)

r2_train = r2_score(y_train, y_train)
r2_test = r2_score(y_test, y_pred)

print("RMSE train:", rmse_train)
print("RMSE test:", rmse_test)

print("R2 train:", r2_train)
print("R2 test:", r2_test)

 A la vista de los resultados obtenidos, hemos decidido hacer la valoración de lo que podría suponer que el modelo no excluya ningún tipo de información, es decir, incluir las variables categóricas también a pesar de que ello pueda consumir más recursos, pero lo buscamos con objeto de ver si de esta manera obtenemos mejores resultados. Y lo hacemos también con Random Forest.

#### Modelos con V. Categóricas

In [302]:
from sklearn.model_selection import train_test_split

df_encoded = pd.get_dummies(restaurants.drop(['Unnamed: 0', 'name'], axis=1), drop_first=True)

X = df_encoded.drop('rate', axis=1)
y = df_encoded['rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [303]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

dt = DecisionTreeRegressor(random_state=0)
rf = RandomForestRegressor(random_state=0, n_jobs=-1)

models = {'Decision Tree': dt, 'Random Forest': rf}

rmse_values = {}
r2_values = {}
# Modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    rmse_values[name] = rmse
    r2_values[name] = r2

rmse_values, r2_values

De esta manera, obtenemos un grandísimo resultado tanto con Árbol de Decisión como con Random Forest, siendo esta la forma más precisa de predecir la valoración de los usuarios. 

## [EJERCICIO 3] Explicación

Disponemos de un conjunto de datos de pasajeros aéreos desde 2015 hasta 2017 y para futuros vuelos, queremos poder predecir los pasajeros que van a volar en base a los datos pasados de pasajeros, a las condiciones de vuelo u otras variables que puedan influir. 

Para ello se pide lo siguiente:

#### a) Visualiza el comportamiento de la serie temporal de pasajeros (npasajeros).
#### b) Visualiza el número de pasajeros en función del tiempo agrupado por mes.
#### c) Realiza una descomposición de la serie temporal para analizar la tendencia y la estacionalidad. ¿Se trata de una serie estacionaria?
#### d) Determina las transformaciones que hace que la serie sea estacionaria. Recuerda que para ello debes prestar atención a la estacionalidad y la tendencia.
#### e) Examina el correlograma y correlograma parcial de la serie. Esto te ayudará a seleccionar los parámetros para el modelo que deberás aplicar en el apartado siguiente.
#### f) Aplica un modelo de series temporales para predecir el número de pasajeros. Para ello divide el dataset en 700 muestras para train y 90 para test.
#### g) Visualiza el resultado en una gráfica en la que se muestren tanto la serie predicha como la serie real.
#### h) Utiliza el rmse para medir el resultado de tu modelo.
#### i) Trata de mejorar los resultados añadiendo variables exógenas al modelo.

##### Deberás entrenar y validar el modelo seleccionado sobre el dataset "datos_pasajeros.csv".

## [EJERCICIO 3] Resolución

In [304]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.stattools as sts

import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error

from statsmodels.tsa.statespace.sarimax import SARIMAX

from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima

warnings.filterwarnings("ignore")

In [305]:
pasajeros= pd.read_csv("datos_pasajeros.csv",sep = ";")
pasajeros.head()

### Preprocesado

In [306]:
pasajeros.shape

In [307]:
print(pasajeros.columns)

In [308]:
columns=["fecha","mes","festivo","npasajeros","intensidad_evento","inten_lluvia","ocupacion_trafico","semana_mes","day_of_week"]

In [309]:
pasajeros.isna().sum()

In [310]:
pasajeros.dtypes

In [311]:
from ydata_profiling import ProfileReport

report = ProfileReport(pasajeros)
report

Pasamos los valores a numéricos

In [312]:
for i in columns:
    pasajeros[i] = pd.to_numeric(pasajeros[i],errors='ignore')

In [313]:
pasajeros.dtypes

Transformamos el texto de la columna fecha en fechas

In [314]:
pasajeros.fecha = pd.to_datetime(pasajeros.fecha, dayfirst = False) 

In [315]:
type(pasajeros.fecha[0])

In [316]:
pasajeros.set_index("fecha", inplace=True)
pasajeros.head(5)

### a) Visualiza el comportamiento de la serie temporal de pasajeros (npasajeros).

In [317]:
import matplotlib.pyplot as plt

pasajeros.npasajeros.plot(figsize=(10,3), title = "Pasajeros")
plt.show()

### b) Visualiza el número de pasajeros en función del tiempo agrupado por mes.

Utilizamos el método resample() de pandas para poder agrupar por distintos intervalos de tiempo. ("M") agrupa las fechas por intervalos de meses

In [318]:
# pasajeros.asfreq("m")

In [319]:
pasajeros_resample = pasajeros.resample("M")["npasajeros"].sum()
pasajeros_resample =pd.DataFrame(pasajeros_resample)
pasajeros_resample.head()

In [320]:
pasajeros_resample.npasajeros.plot(figsize=(10,3), title = "Pasajeros/mes")
plt.show()

In [321]:
sns.displot(pasajeros_resample)

### c) Realiza una descomposición de la serie temporal para analizar la tendencia y la estacionalidad. ¿Se trata de una serie estacionaria?

In [322]:
descomposicion_aditiva_mes = seasonal_decompose(pasajeros_resample["npasajeros"], model= "additive")
descomposicion_aditiva_mes.plot()
plt.show()

In [323]:
descomposicion_aditiva = seasonal_decompose(pasajeros["npasajeros"], model= "additive")
descomposicion_aditiva.plot()
plt.show()

- Test de hipótesis: Obtenemos que el p valor es mayor a 0.05 por lo que la serie no parece estacionaria.

In [324]:
sts.adfuller(pasajeros.npasajeros)

- QQplot

In [325]:
scipy.stats.probplot(pasajeros.npasajeros, plot =  plt)
#le pasamos la variable que queremos visualizar y el tipo de gráfico (matplotlib)
plt.title("QQ Plot", size = 15) 
plt.show()

### d) Determina las transformaciones que hace que la serie sea estacionaria. Recuerda que para ello debes prestar atención a la estacionalidad y la tendencia.

-Tras analizar la gráfica de la estacionalidad por días, la serie no es estacional. En el caso de analizarla por meses sí sería estacional.

Test de Dickey Fuller

In [326]:
sts.adfuller(descomposicion_aditiva.trend.dropna())

In [327]:
sts.adfuller(pasajeros.npasajeros.diff()[1:])

In [328]:
plt.figure(figsize=(20,10))
plt.plot(pasajeros.npasajeros.diff()[1:])
plt.title("Plot for lagged PercentOfBaseline Vs Time for Chile")
plt.show()

La tendencia con **una diferenciación** es estacionaria.  **d=1**

### e) Examina el correlograma y correlograma parcial de la serie. Esto te ayudará a seleccionar los parámetros para el modelo que deberás aplicar en el apartado siguiente.

**Aplicamos el modelo ARIMA al carecer de estacionalidad**

**Función de autocorrelación (cálculo de q)**

In [329]:
sgt.plot_acf(pasajeros.npasajeros.diff()[1:],
             zero = False)
plt.title("Autocorrelación", size = 12)
plt.show()

**q =5**

**Función de autocorrelación parcial (cálculo de p)**

In [330]:
sgt.plot_pacf(pasajeros.npasajeros.diff()[1:],
              zero = False, method = ('ols'))
plt.title("Autocorrelación parcial", size = 12)
plt.show()

**p =7**

### f) Aplica un modelo de series temporales para predecir el número de pasajeros. Para ello divide el dataset en 700 muestras para train y 90 para test.

- División entrenamiento y test:

In [331]:
print(len(pasajeros))

train = pasajeros[["npasajeros"]].iloc[:700]
test = pasajeros[["npasajeros"]].iloc[700:]
print(len(train))
print(len(test))

In [332]:
train

***Definición del modelo ARIMAX:***
El modelo será p = 7, d = 1, q = 5.

Vamos a usar ahora a usar el modelo autoarima para encontrar los valores más óptimos de p y q y ver si coinciden aproximadamente con los calculados.

In [333]:
arima_model =  auto_arima(train,start_p=0, d=1, start_q=0,
                          max_p=9, max_q=7, seasonal=False,
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 50 )

Una vez encontrado el mejor modelo, vamos a seleccionarlo y entrenar un modelo ARIMA con él:

In [334]:
modelArima = ARIMA(train, order = (7,1,7))
resultados = modelArima.fit()

### g) Visualiza el resultado en una gráfica en la que se muestren tanto la serie predicha como la serie real.

In [335]:
inicio_prediccion = len(train)
fin_prediccion = len(test)
test["prediccion"] = resultados.predict(
    start=inicio_prediccion,end=inicio_prediccion + fin_prediccion)

In [336]:
plt.figure(figsize=(20, 6))
plt.plot(train)
plt.plot(test["npasajeros"]) 
plt.plot(test["prediccion"])
plt.xticks(rotation = 90)
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

### h) Utiliza el rmse para medir el resultado de tu modelo.

In [337]:
MSE = mean_squared_error(test["npasajeros"],test["prediccion"])
RMSE = np.sqrt(MSE)
print("Root Mean Square Error:", RMSE)

### i) Trata de mejorar los resultados añadiendo variables exógenas al modelo.

Consideramos que las variantes exógenas que más pueden afectar son la intensidad de lluvia y la ocupación de tráfico

In [338]:
pasajeros.describe()

Introducimos la variable exógena: inten_lluvia

In [339]:
modelArima = ARIMA(train,
                   exog = pasajeros[["inten_lluvia"]].iloc[:700],
                   order = (7,1,7))

resultados = modelArima.fit()

In [340]:
inicio_prediccion = len(train)
fin_prediccion = len(test)
test["prediccion"] = resultados.predict(
    start=inicio_prediccion,end=inicio_prediccion + fin_prediccion -1,exog = pasajeros[["inten_lluvia"]].iloc[700:])

In [341]:
plt.figure(figsize=(20, 6))
plt.plot(train)
plt.plot(pasajeros["inten_lluvia"])
plt.plot(test["npasajeros"]) 
plt.plot(test["prediccion"])
plt.xticks(rotation = 90)
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

In [342]:
MSE = mean_squared_error(test["npasajeros"],test["prediccion"])
RMSE = np.sqrt(MSE)
print("Root Mean Square Error con la variable éxogena intensidad de lluvia:", RMSE)

- Introducimos la variable exógena: ocupacion_trafico

In [343]:
modelArima = ARIMA(train,
                   exog = pasajeros[["ocupacion_trafico"]].iloc[:700],
                   order = (7,1,7))

resultados = modelArima.fit()

In [344]:
inicio_prediccion = len(train)
fin_prediccion = len(test)
test["prediccion"] = resultados.predict(
    start=inicio_prediccion,end=inicio_prediccion + fin_prediccion -1,exog = pasajeros[["ocupacion_trafico"]].iloc[700:])

In [345]:
plt.figure(figsize=(20, 6))
plt.plot(train)
plt.plot(pasajeros["ocupacion_trafico"])
plt.plot(test["npasajeros"]) 
plt.plot(test["prediccion"])
plt.xticks(rotation = 90)
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

In [346]:
MSE = mean_squared_error(test["npasajeros"],test["prediccion"])
RMSE = np.sqrt(MSE)
print("Root Mean Square Error con la variable éxogena ocupación de tráfico:", RMSE)

**Conclusiones:**
Al introducir la variable exógena disminuye el error en la predicción